In [ ]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import get_max_db_id
from util import collect_segments
from util import download_audio_gdrive

In [79]:
tsv_location = "/home/monlamai/Downloads/adata"

In [80]:
group_ab_ga_id = 1
group_ab_gb_id = 2
group_ab_gc_id = 7

In [81]:
import pandas as pd

In [82]:
from pathlib import Path
df_array = []
for path in Path(tsv_location).rglob('*.tsv'):
    # print(path)
    df_temp = pd.read_csv(path, sep='\t')
    df_array.append(df_temp)
df = pd.concat(df_array, ignore_index=True)

In [83]:
df.shape

(8335, 6)

In [84]:
df.head()

,Unnamed: 0,file_name,url,inference_transcript,audio_duration,group
0,0,STT_AB00609_0144_816749_to_825435,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,14 ཞེ་འཁམ་པ་ནི་ཞེ་ལ་བརྩུབ་པའི་སེམས་ཀར་ཀར་བ་སྟེ...,8.686,STTAB00609
1,98,STT_AB00609_0056_313319_to_317022,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ར་མ་ཟད་འཇིག་རྟེན་ཕ་རོལ་གྱི་དོན་དང་ཡང་ནི་,3.703,STTAB00609
2,313,STT_AB00609_0096_546646_to_550179,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,འགལ་བར་འགྱུར་ཏེ།ཁོང་ཁྲོ་ལ་སྐབས་བྱིན་པས་ཤི་བའི་...,3.533,STTAB00609
3,321,STT_AB00609_0121_687226_to_692329,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ད་དུ་མི་འོང་བ་འཕེན་པའི་ཕྱིར་རོ།ᨎགཉིས་པ།ཕྱིས་ཀྱ...,5.103,STTAB00609
4,357,STT_AB00609_0015_90008_to_94513,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སྔལ་མི་འདོད་པ་དང་གནོད་ལན་བྱེད་པ་གཉིས་འགལ་བར་བསྟ,4.505,STTAB00609


In [85]:
df['group_id'] = group_ab_ga_id

In [86]:
df['state'] = 'imported'

In [87]:
df.fillna('', inplace=True)

In [88]:
def filter_length(st):
    return len(st) < 500

In [89]:
df = df[df['inference_transcript'].apply(lambda x: len(x) < 500)]

In [91]:
df.head()

,Unnamed: 0,file_name,url,inference_transcript,audio_duration,group,group_id,state
0,0,STT_AB00609_0144_816749_to_825435,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,14 ཞེ་འཁམ་པ་ནི་ཞེ་ལ་བརྩུབ་པའི་སེམས་ཀར་ཀར་བ་སྟེ...,8.686,STTAB00609,1,imported
1,98,STT_AB00609_0056_313319_to_317022,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ར་མ་ཟད་འཇིག་རྟེན་ཕ་རོལ་གྱི་དོན་དང་ཡང་ནི་,3.703,STTAB00609,1,imported
2,313,STT_AB00609_0096_546646_to_550179,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,འགལ་བར་འགྱུར་ཏེ།ཁོང་ཁྲོ་ལ་སྐབས་བྱིན་པས་ཤི་བའི་...,3.533,STTAB00609,1,imported
3,321,STT_AB00609_0121_687226_to_692329,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ད་དུ་མི་འོང་བ་འཕེན་པའི་ཕྱིར་རོ།ᨎགཉིས་པ།ཕྱིས་ཀྱ...,5.103,STTAB00609,1,imported
4,357,STT_AB00609_0015_90008_to_94513,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སྔལ་མི་འདོད་པ་དང་གནོད་ལན་བྱེད་པ་གཉིས་འགལ་བར་བསྟ,4.505,STTAB00609,1,imported


In [92]:
last_db_id = get_max_db_id()

id_arr = list(range(last_db_id + 1, df.shape[0] + last_db_id + 1))

df['id'] = id_arr

The maximum ID in the 'Task' table is: 420828


In [93]:
df = df[['id', 'group_id', 'file_name', 'url', 'inference_transcript', 'audio_duration', 'state']]

In [94]:
df.shape

(8334, 7)

In [96]:
df['file_name'].str[0:11].unique()

array(['STT_AB00609', 'STT_AB00608', 'STT_AB00614', 'STT_AB00596',
       'STT_AB00636', 'STT_AB00627', 'STT_AB00610', 'STT_AB00617',
       'STT_AB00630', 'STT_AB00602', 'STT_AB00615', 'STT_AB00639',
       'STT_AB00607', 'STT_AB00629', 'STT_AB00632', 'STT_AB00601',
       'STT_AB00642', 'STT_AB00628', 'STT_AB00599', 'STT_AB00611',
       'STT_AB00622', 'STT_AB00624', 'STT_AB00631', 'STT_AB00634',
       'STT_AB00626', 'STT_AB00641', 'STT_AB00605', 'STT_AB00638',
       'STT_AB00640', 'STT_AB00618', 'STT_AB00597', 'STT_AB00643',
       'STT_AB00613', 'STT_AB00625', 'STT_AB00612', 'STT_AB00598',
       'STT_AB00635', 'STT_AB00637', 'STT_AB00606', 'STT_AB00633',
       'STT_AB00600', 'STT_AB00604'], dtype=object)

In [97]:
df.to_csv('stt_ab_ga.csv', index=False)

In [98]:
import pandas as pd
df = pd.read_csv('stt_ab_from_gdrive.tsv', sep='\t')

In [99]:
df.head()

,file_name,url,inference_transcript,audio_duration
0,STT_AB00609_0144_816749_to_825435,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,གཉིས་པའི་དོན་ཀྱང་སེམས་བསྐྱེད་ལ་བརྟེན་ནས། ལས་ཉོ...,8.686
1,STT_AB00611_0008_40213_to_45332,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,བརྩོན་འགྲུས་དང་། དྲན་པ་དང་། ཏིང་ངེ་འཛིན་དང་། ཤ...,5.119
2,STT_AB00619_0144_804991_to_808558,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,བློའི་ཡུལ་ཀུན་རྫོབ་པར་འདོད་པའི་དོན་དེ་བཞིན་དུ་...,3.567
3,STT_AB00604_0095_556646_to_565110,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཐེག་དམན་གྱི་མདོར་ཆོས་ཐམས་ཅད་བདག་མེད་པར་གསུང་པའ...,8.464
4,STT_AB00629_0188_1092261_to_1099513,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,གླང་པོ་ཆེ་ལ་སོགས་པའི་གཟུགས་མིག་གི་སྐྱེད་སྤྱེད་...,7.252


In [100]:
last_db_id = get_max_db_id()

id_arr = list(range(last_db_id + 1, df.shape[0] + last_db_id + 1))

df['id'] = id_arr

The maximum ID in the 'Task' table is: 420928


In [101]:
df['state'] = 'imported'
df['group_id'] = group_ab_ga_id

In [103]:
df.head()

,file_name,url,inference_transcript,audio_duration,id,state,group_id
0,STT_AB00609_0144_816749_to_825435,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,གཉིས་པའི་དོན་ཀྱང་སེམས་བསྐྱེད་ལ་བརྟེན་ནས། ལས་ཉོ...,8.686,420929,imported,1
1,STT_AB00611_0008_40213_to_45332,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,བརྩོན་འགྲུས་དང་། དྲན་པ་དང་། ཏིང་ངེ་འཛིན་དང་། ཤ...,5.119,420930,imported,1
2,STT_AB00619_0144_804991_to_808558,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,བློའི་ཡུལ་ཀུན་རྫོབ་པར་འདོད་པའི་དོན་དེ་བཞིན་དུ་...,3.567,420931,imported,1
3,STT_AB00604_0095_556646_to_565110,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཐེག་དམན་གྱི་མདོར་ཆོས་ཐམས་ཅད་བདག་མེད་པར་གསུང་པའ...,8.464,420932,imported,1
4,STT_AB00629_0188_1092261_to_1099513,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,གླང་པོ་ཆེ་ལ་སོགས་པའི་གཟུགས་མིག་གི་སྐྱེད་སྤྱེད་...,7.252,420933,imported,1


In [104]:
df.to_csv('stt_ab_ga_not_transfered.csv', index=False)